In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import random
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import DeepTreeEnsemble
import torch.nn.functional as F
from importlib import reload
import pandas as pd
reload(DeepTreeEnsemble)
from DeepTreeEnsemble import DeepTreeEnsemble, create_dataloaders_for_dataset 
# if this ^ doesn't work, try 'pip install -e .' in the root directory of it. Make sure you are in 
# the desired environment. 

In [9]:
train_dataloader, test_dataloader, input_size, output_size = create_dataloaders_for_dataset(dataset_name='breast_cancer', task='classification', batch_size=32)

xtrain size 455
xtest size 114
y_train size 455
y_test size 114


In [10]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, dropout=.5):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Create the first hidden layer
        self.layers.append(nn.Linear(input_size, hidden_layers[0]))
        self.batch_norms.append(nn.BatchNorm1d(hidden_layers[0]))

        # Create additional hidden layers
        for i in range(1, len(hidden_layers)):
            self.layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            self.batch_norms.append(nn.BatchNorm1d(hidden_layers[i]))

        # Output layer
        self.output = nn.Linear(hidden_layers[-1], output_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        i = 0
        for layer, batch_norm in zip(self.layers, self.batch_norms):
            # x = F.relu(batch_norm(layer(x)))
            i=i+1
            x = self.dropout(x)
            # print('Layer dtype: ', layer.weight.dtype)
            x = x.to(torch.float32) # <-- area of concern
            # print('tensor dytpe: ', x.dtype)
            # layer.weight = layer.weight.to(torch.float32)
            x = F.relu(batch_norm(layer(x)))
            
            # print(f'dtype is {x.dtype} after layer {i}')

        x = self.output(x)
        return x

In [11]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp4b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [12]:
def run_experiment(DTE, trails, output_directory):
    results = pd.DataFrame()

    for i in range(trails):
        df1 = DTE.train_DTE()
        df2 = DTE.single_model()
        combined_df = pd.concat([df1, df2], axis=1)
        results = pd.concat([results, combined_df], axis=0)

    results.to_csv(output_directory, index=False)

In [13]:
output_directory = './output.csv'
run_experiment(DTEmlp4b, trails=5, output_directory=output_directory)

Randomly initializing 4 models
Training Model with id 1
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 12.14it/s]


Total Epochs: 10
Train loss start: 1.0790342092514038, Train loss end: 1.052383303642273
Highest Mean Train Accuracy (over epoch): 0.3975892857142857
Test loss start: 1.0830665826797485, Test loss end: 1.030713677406311
Highest mean Test Accuracy (over epoch): 0.39262152777777776
 Test F1 score at highest accuracy: 0.3594549651297133
./saved_models/model_1.pth 

Training Model with id 2
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 20.32it/s]


Total Epochs: 10
Train loss start: 1.0080317258834839, Train loss end: 0.9729445576667786
Highest Mean Train Accuracy (over epoch): 0.5372321428571428
Test loss start: 1.0952141284942627, Test loss end: 1.0682154893875122
Highest mean Test Accuracy (over epoch): 0.40503472222222214
 Test F1 score at highest accuracy: 0.5276389410515813
./saved_models/model_2.pth 

Training Model with id 3
in multi


...

Total Epochs: 10
Train loss start: 0.9441336393356323, Train loss end: 0.9405580163002014
Highest Mean Train Accuracy (over epoch): 0.6350892857142857
Test loss start: 0.9765018820762634, Test loss end: 1.0124828815460205
Highest mean Test Accuracy (over epoch): 0.6390625
 Test F1 score at highest accuracy: 0.5705670241300325
./saved_models/model_8.pth 

Training Model with id 6
in multi


Training Process: 100%|██████████| 10/10 [00:00<00:00, 15.39it/s]


Total Epochs: 10
Train loss start: 0.8539381623268127, Train loss end: 0.7479490041732788
Highest Mean Train Accuracy (over epoch): 0.6909523809523809
Test loss start: 0.9430342316627502, Test loss end: 0.9682461619377136
Highest mean Test Accuracy (over epoch): 0.6765625
 Test F1 score at highest accuracy: 0.6445920261057289
./saved_models/model_6.pth 

 Maximum accuracy: 0.6765625, F1: 0.6445920261057289, Test: 0.924060371518135, Train: 0.888720017671585
in multi


Training Process: 100%|██████████| 70/70 [00:06<00:00, 10.79it/s]

Total Epochs: 70
Train loss start: 1.1321748495101929, Train loss end: 0.7479714751243591
Highest Mean Train Accuracy (over epoch): 0.7585586734693879
Test loss start: 1.039901614189148, Test loss end: 0.7266463041305542
Highest mean Test Accuracy (over epoch): 0.818390376984127
 Test F1 score at highest accuracy: 0.7644566259957994


In [8]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp8b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [4]:
output_directory = './outputBase8.csv'
run_experiment(DTEmlp8b, trails=5, output_directory=output_directory)

NameError: name 'DTEmlp8b' is not defined